In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

import time

In [11]:
fold = 0



train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

# Split the data into features and target
target_features = ['NObeyesdad']

In [12]:
# Split the data into features and target
X_train = train.drop(['id'] + target_features, axis=1)
X_test = test.drop(['id'], axis=1)

y_train = train[target_features].values

In [13]:
# Reshape the target arrays
y_train = y_train.reshape(-1)


# Initialize label encoder
label_encoder = LabelEncoder()

# Encode the target labels
y_train = label_encoder.fit_transform(y_train)


# Define features
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
numerical_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [14]:
# Create a column transformer for one-hot encoding and standard scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Create a pipeline with the preprocessor and the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier())
])

In [15]:
start = time.time()

# Fit the model
pipeline.fit(X_train, y_train)

# make predictions
preds = pipeline.predict(X_test)

end = time.time()
time_taken = end - start

# Convert predictions back to original labels
preds_labels = label_encoder.inverse_transform(preds)

# Calculate the accuracy on train
train_preds = pipeline.predict(X_train)
accuracy = accuracy_score(y_train, train_preds)
print(f"Accuracy={accuracy}")

print(f"Fold={fold}, Accuracy = {accuracy:.4f}, Time Taken={time_taken:.2f}sec")

Accuracy=0.9816456306002505
Fold=0, Accuracy = 0.9816, Time Taken=1.00sec


In [16]:
# Prepare the submission dataframe
submission = pd.DataFrame({
    'id': test['id'],
    'NObeyesdad': preds_labels
})

In [17]:
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [18]:
# Save the submission file
# submission.to_csv('submission.csv', index=False)
print("Submission is successfully saved!")

Submission is successfully saved!
